## Datashader dashboard

This notebook contains the code for an interactive dashboard for making [Datashader](http://datashader.org) plots from any dataset. Apart from Datashader itself, the code relies on other Python packages from the [PyViz](http://pyviz.org) project that are each designed to make it simple to:

- lay out plots and widgets into an app or dashboard, in a notebook or for serving separately ([Panel](http://panel.pyviz.org))
- build interactive web-based plots without writing JavaScript ([Bokeh](http://bokeh.pydata.org))
- build interactive Bokeh-based plots backed by Datashader, from concise declarations ([HoloViews](http://holoviews.org))
- express dependencies between parameters and code to build reactive interfaces declaratively ([Param](http://param.pyviz.org))
- describe the fields and plotting information needed to plot a dataset, in a text file ([Intake](http://intake.readthedocs.io))

In [ ]:
import os, param, colorcet, holoviews as hv, panel as pn
import intake, geoviews.tile_sources as gts
from holoviews.operation.datashader import rasterize, shade, dynspread
hv.extension('bokeh', logo=False)

You can run the dashboard here in the notebook by editing the `data_source` below to specify some dataset defined in `dashboard.yml`, or else you can launch a separate server process in a new browser tab with a command like:

```
DS_DATASET=nyc_taxi panel serve --show dashboard.ipynb
DS_DATASET=census   panel serve --show dashboard.ipynb
DS_DATASET=opensky  panel serve --show dashboard.ipynb
DS_DATASET=osm      panel serve --show dashboard.ipynb
```

To launch multiple dashboards at once, you'll need to add `-p 5001` (etc.) to select a unique port number for the web page to use for communicating with the Bokeh server.  Otherwise, be sure to kill the server process before launching another instance.

For most of these datasets, if you have less than 16GB of RAM on your machine, you should remove the `.persist()` call below, to tell [Dask](http://dask.pydata.org) to work out of core instead of loading all data into memory.  However, doing so will make interactive use substantially slower than if sufficient memory were available.

In [ ]:
data_source = os.getenv("DS_DATASET", "nyc_taxi")
cat = intake.open_catalog('dashboard.yml')
source = getattr(cat, data_source)
source.to_dask().persist();

The Intake `source` object lets us treat data in many different formats the same in the rest of the code here. We can now build a class that captures some parameters that the user can vary along with how those parameters relate to the code needed to update the displayed plot of that data source:

In [ ]:
import numpy as np

In [ ]:
gopts = hv.opts.WMTS(width=800, height=500, xaxis=None, yaxis=None, bgcolor='black', show_grid=False)
cmaps = {n: colorcet.palette[n] for n in ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']}
norms = {'Histogram_Equalization': 'eq_hist', 'Linear': 'linear', 'Log': 'log', 'Cube root': 'cbrt'}
maps  = ['CartoMidnight', 'StamenWatercolor', 'StamenTonerBackground', 'EsriImagery', 'EsriUSATopo', 'EsriTerrain']
bases = {name: ts.relabel(name) for name, ts in gts.tile_sources.items() if name in maps}

class Explorer(param.Parameterized):
    opacity = param.Magnitude(default=0.75, doc="Alpha value for the data opacity")
    basemap_opacity = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    cmap = param.ObjectSelector(cmaps['fire'], objects=cmaps)
    basemap = param.ObjectSelector(bases['EsriImagery'], objects=bases)
    show_labels = param.Boolean(default=True)
    normalization = param.ObjectSelector(default='eq_hist', objects=norms)
    spread_size = param.Integer(0, bounds=(0, 10))
    plot = param.ObjectSelector(default=source.plots[0], objects=source.plots)
    
    @param.depends('plot')
    def points(self):
        plot_method = getattr(source.plot, self.plot)
        points = plot_method() # could add extra hvplot kwargs here
        return points

    @param.depends('basemap_opacity','basemap')
    def tiles(self):
        return self.basemap.opts(gopts).opts(alpha=self.basemap_opacity)

    @param.depends('show_labels')
    def labels(self):
        return gts.StamenLabels.options(level='annotation', alpha=1 if self.show_labels else 0)
    
    @param.depends('opacity')
    def apply_opacity(self, shaded):
        return shaded.opts(alpha=self.opacity, show_legend=False)

    def view(self,**kwargs):
        points = hv.DynamicMap(self.points)
        agg    = rasterize(points, x_sampling=1, y_sampling=1, width=600, height=400)
        shade_stream = hv.streams.Params(self, ['cmap', 'normalization'])
        spread_stream = hv.streams.Params(self, ['spread_size'], rename={'spread_size': 'max_px'})
        tiles  = hv.DynamicMap(self.tiles)
        labels = hv.DynamicMap(self.labels)
        shaded = dynspread(shade(agg, streams=[shade_stream]), streams=[spread_stream])
        shaded = hv.util.Dynamic(shaded, operation=self.apply_opacity)
        return tiles * shaded * labels

explorer = Explorer(name="")
pn.Row(pn.Column("# Datashader", pn.Param(explorer.param, expand_button=False)), explorer.view()).servable()

If you are viewing this notebook with a live Python server process running, adjusting one of the widgets above should update the plot, re-running only the code needed to update that particular item without re-running datashader if that's not needed. It should work the same when launched as a separate server process, but without the extra text and code visible as in this notebook. Here the `.servable()` method call indicates what should be served when run separately, and you can just copy the code out of this notebook into a .py file that will work the same as this .ipynb file.